In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformers import pipeline

In [3]:
data = pd.read_csv('/content/news.csv')


/tmp/ipython-input-3-541212611.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/news.csv')


In [4]:
print(data.head())

  Unnamed: 0                                              title  \
0       8476                       You Can Smell Hillary’s Fear   
1      10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2       3608        Kerry to go to Paris in gesture of sympathy   
3      10142  Bernie supporters on Twitter erupt in anger ag...   
4        875   The Battle of New York: Why This Primary Matters   

                                                text label Unnamed: 4  \
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE        NaN   
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE        NaN   
2  U.S. Secretary of State John F. Kerry said Mon...  REAL        NaN   
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE        NaN   
4  It's primary day in New York and front-runners...  REAL        NaN   

  Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 131  \
0        NaN        NaN        NaN        NaN        NaN  ...          NaN   
1 

In [5]:
X = data['text']  # Headlines (features)
y = data['label'] # Real/Fake labels (target)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # Added train_test_split

# Handle missing values in 'text' and 'label' columns before vectorization
X_train = X_train.fillna('')  # Replace NaN values with empty strings in text column
X_test = X_test.fillna('')  # Replace NaN values with empty strings in text column

# Drop rows with missing values in the target variable and align X_train/X_test
y_train = y_train.dropna()
y_test = y_test.dropna()

X_train = X_train[y_train.index]  # Align X_train with cleaned y_train
X_test = X_test[y_test.index]  # Align X_test with cleaned y_test


# Reset the index after dropping rows (optional but recommended for clean indexing)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Apply TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [7]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)



LogisticRegression()

In [8]:
y_pred = model.predict(X_test_tfidf)


In [9]:
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8685376661742984


In [10]:
def classify_headline(headline):
    classifier = pipeline("zero-shot-classification")
    result = classifier(headline, candidate_labels=["real", "fake"])

    # Classifying the headline as True for real and False for fake
    if result['labels'][0] == 'real':
        print(f"Prediction: True (Real News)")
        return True
    else:
        print(f"Prediction: False (Fake News)")
        return False



In [11]:
headline = input("Enter a news headline: ")
result = classify_headline(headline)

Enter a news headline: "AI is Revolutionizing the Future: From Automation to Innovation"


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Prediction: True (Real News)
